In [1]:
import json
import pprint as pp
from IPython.display import Audio, clear_output
from ibm_watson import TextToSpeechV1, AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [2]:
def get_text_to_speech(TtS_api_key='NlhclhDNYCgBT0iAZMsvGfhqadWAXiv0emYqLBJEKmkG', 
                       TtS_service_url='https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/09e04cc2-d35f-4bf5-b98f-1c869434f691'):
    TtS_authenticator = IAMAuthenticator(TtS_api_key)
    TtS = TextToSpeechV1(authenticator=TtS_authenticator)
    TtS.set_service_url(TtS_service_url)
    return TtS

In [3]:
def gen_audio_from_text(text_str=f'Text to speech is working', audio_file_name=f'speech_temp.mp3', TtS=get_text_to_speech()):
    with open(f'{audio_file_name}', 'wb') as audio_file:
        res = TtS.synthesize(text_str, accept='audio/mp3', voice='en-US_AllisonV3Voice').get_result()
        audio_file.write(res.content)
    clear_output()
    display(Audio('speech_temp.mp3', autoplay=True))

In [4]:
def get_assistant_and_session_id(assistant_api_key = '1vbLLsF7S237KdlIAvNxJXFmt4nPGn_YIujYUKUYWuan',
                  assistant_service_url = 'https://api.us-south.assistant.watson.cloud.ibm.com/instances/39a5994d-8151-4f52-932c-8f7496ae0945',
                  assistant_version = '2022-03-21',
                  assistant_id = '2e0ecccd-8076-444e-8488-282b3dd64404'):
    
    assistant_authenticator = IAMAuthenticator(assistant_api_key)
    assistant_obj = AssistantV2(version=assistant_version, authenticator=assistant_authenticator)
    assistant_obj.set_service_url(assistant_service_url)
    
    session_body = assistant_obj.create_session(assistant_id).get_result()
    session_id = session_body['session_id']
    
    return_dict = {
        'assistant_obj': assistant_obj,
        'assistant_id': assistant_id, 
        'session_id': session_id
    }    
    return return_dict

In [5]:
def send_msg_and_get_res_text(message_text, assistant_dict):
    assistant_obj = assistant_dict['assistant_obj']
    assistant_id = assistant_dict['assistant_id']
    session_id = assistant_dict['session_id']
    message_body = {'text': message_text}
    
    print(f'... waiting for response ...')
    response_body = assistant_obj.message(assistant_id=assistant_id,
                                          session_id=session_id,
                                          input=message_body).get_result()
    response_texts = [res['text'] for res in response_body["output"]["generic"]]
    res_concatenated = '\n'.join(response_texts)
    res_to_be_read = res_concatenated.replace('\n', ' ')
    
    return res_concatenated, res_to_be_read

In [6]:
def get_assistant_output(message_text, assistant_dict, TtS):
    res_concatenated, res_to_be_read = send_msg_and_get_res_text(message_text=message_text, 
                                                                 assistant_dict=assistant_dict)
    
    gen_audio_from_text(text_str=res_to_be_read, TtS = TtS)
    print(f'{res_concatenated}', flush=True)
#     print(f'{res_to_be_read}')

In [7]:
def interact_with_assistant(assistant_dict, TtS):
    message_text = ''
    while True:
        get_assistant_output(message_text, assistant_dict, TtS)
        if message_text == 'goodbye':
            break
        message_text = input(">>> ")
        
        

In [8]:
TtS = get_text_to_speech()
assistant_dict = get_assistant_and_session_id()
interact_with_assistant(assistant_dict, TtS)

We have standard drinks:
* Cappuccino * Latte
We also have customized drinks:
* Irish * Mocha * Flat White
What coffee do you like to order today?


KeyboardInterrupt: Interrupted by user